### Setup

In [1]:
import glob
import io
import math
import os

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import style
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
style.use('ggplot')

/usr/local/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load Dataset and Randomization

In [3]:
mnist_dataframe = pd.read_csv(
    io.open("mnist_train_small.csv", "r"),
    sep=",",
    header=None)
mnist_dataframe = mnist_dataframe.head(10000) # first 10000 rows
mnist_dataframe = mnist_dataframe.reindex(np.random.permutation(mnist_dataframe.index))
mnist_dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
7697,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
895,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6604,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
mnist_dataframe.loc[2:25, 71:75]

,71,72,73,74,75
2,0,0,0,0,0
4441,0,0,0,0,0
3845,18,255,81,0,0
8122,0,0,0,0,0
7224,0,0,0,0,0
...,...,...,...,...,...
4385,0,0,0,0,0
7663,0,0,0,0,0
3757,0,0,0,0,0
6525,0,0,0,0,0
